In [10]:
import pandas as pd
import pickle
import numpy as np

# Load the bar review dataset 
review = pd.read_pickle('../output/bar_reviews_cleaned_and_tokenized_SF.pickle')
print review.head(2)
print review.tail(2)

               business_id        date               review_id stars  \
10  UsFtqoBl7naz8AVUBZMjQQ  2013-11-08  Di3exaUCFNw1V4kSNW5pgA     5   
11  UsFtqoBl7naz8AVUBZMjQQ  2014-03-29  0Lua2-PbqEQMjD9r89-asw     3   

                                                 text    type  \
10  All the food is great here. But the best thing...  review   
11  We checked this place out this past Monday for...  review   

                   user_id  votes_cool  votes_funny  votes_useful  \
10  uK8tzraOp4M5u3uYrqIBXg         0.0          0.0           0.0   
11  I_47G-R2_egp7ME5u_ltew         0.0          0.0           0.0   

                                    cleaned_tokenized  
10  [[food, great], [best, thing, wing], [wing, si...  
11  [[checked, place, past, monday, wing, night], ...  
                                    business_id date  \
84215  8781c06a4e2407f5e027cd503f4aab675e76615b  NaN   
84216  8781c06a4e2407f5e027cd503f4aab675e76615b  NaN   

                                  review_i

In [11]:
# Load the training users
user_set_training = pickle.load(open('../output/training_users.pickle', 'rb'))
# Make the active review set training only 
review = review[review.user_id.isin(user_set_training)]


In [12]:
from collections import OrderedDict
from itertools import chain


# n_reviews = 100 # all of them... 
# Flatten the reviews, so each review is just a single list of words.
reviews_merged_bus = OrderedDict()
business_set = list(set(review.business_id.values[:]))
for i_bus, bus_id in enumerate(business_set):
    if ((i_bus%5)==0):
        print '\r Fraction Processed',float(i_bus+1)/len(business_set),
    # This horrible line first collapses each review of a corresponding business into a list
    # of lists, and then collapses the list of sentences to a long list of words
    reviews_merged_bus[bus_id] = list(chain.from_iterable(chain.from_iterable( 
                                     review.cleaned_tokenized[review.business_id==bus_id] )) )
docs_bus = reviews_merged_bus.values()
print 
print len(docs_bus)

 Fraction Processed 0.999781992588
4587


In [13]:
import gensim
from itertools import chain
import sys
sys.path.append('../vectorsearch/')
import nltk_helper
import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd
n_epochs = 10
n_docs = 10 # -1 for almost all of them...

# Generate the tagged document list. 

docs = [TaggedDocument(words, [business_set[index],])
                             for index, words in enumerate(docs_bus[:])]

print '\nFirst Doc: \n-----------------\n', docs[0]



First Doc: 
-----------------
TaggedDocument(['apparently', 'place', 'get', 'pitcher', 'beer', 'very', 'late', 'sunday', 'evening', 'isnt', 'filled', 'drunk', 'karaoke', 'bad', 'house', 'music', 'dont', 'get', 'wrong', 'love', 'karaoke', 'dancing', 'like', 'go', u'place', 'actually', 'hear', 'person', 'next', 'place', 'run', 'people', 'went', 'high', 'school', 'might', 'want', 'make', 'sure', 'wearing', 'matching', u'sock', 'could', 'embarrassing', 'bartender', 'really', 'feel', 'badly', 'forgetting', 'name', 'super', 'nice', 'let', u'u', 'stay', 'finish', u'drink', 'shoot', 'shit', 'much', 'area', 'changed', 'good', '20', u'min', 'past', 'close', 'busy', 'enough', 'dont', 'feel', 'awkward', 'drinking', 'yet', 'not', 'busy', 'cant', 'get', 'seat', 'good', 'service', u'drink', 'affordable', 'really', 'cant', 'complain', u'the-only-place', u'a-pitcher', u'the-person', u'a-place', u'high-school', u'her-name', u'our-drinks', u'the-shit', u'this-area', u'a-good-20-mins', u'a-seat', u'good-

In [14]:
from gensim.models import doc2vec

model = doc2vec.Doc2Vec(min_count=5, window=6, size=100, sample=1e-4, negative=5, workers=12)
# Build the vocab from list of sentences.
model.build_vocab(docs) 



In [15]:
from random import shuffle

for epoch in range(10):
    print '\rTraining Epoch %i, alpha %1.4f'%(epoch+1, model.alpha),
    #model.train(np.random.permutation(docs))
    shuffle(docs)
    model.train(docs)
    model.alpha -= 0.001 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no decay

#model.init_sims(replace=True)    
# # Normalize the word vectors.
# vec_norms = np.sqrt(np.sum(model.syn0**2, axis=1))
# model.syn0 = (model.syn0/vec_norms[:, numpy.newaxis])
# # Normalize the doc vectors.
# vec_norms = np.sqrt(np.sum(model.docvecs.doctag_syn0**2, axis=1))
# model.docvecs.doctag_syn0 = (model.docvecs.doctag_syn0/vec_norms[:, numpy.newaxis])

model.save('../output/doc2vec_bars_100.model')

Training Epoch 10, alpha 0.0160


In [13]:
# Can find similar documents..
print model.docvecs.most_similar(positive=['KUinHkKyGhznElgIzx0yIw']), '\n'

# Can find similar words...Re: Dream Companies and contact from recruiters
print model.most_similar(positive=['beer']), '\n'

# Can find documents that are most similar to keywords.... 
print model.docvecs.most_similar(positive=[model['beer'], model['music']]), '\n'

# Can find words that are most common in documents
print review.text[review.review_id=='KUinHkKyGhznElgIzx0yIw'].values
print model.most_similar(positive=[model.docvecs['KUinHkKyGhznElgIzx0yIw']]), '\n'


[(u'yyUJKvG-C4VipITrAS0nIQ', 0.6710823774337769), (u'g3fipTPN2LBe_U42niTDcw', 0.6688743829727173), (u'ANaGwB8tVAc1qM1QAJecsQ', 0.6611914038658142), (u'LVjRN5pMJ8hhDmX0lbclpQ', 0.6471917629241943), (u'YP-sxa8i95v_scvXN2o4_w', 0.6342615485191345), (u'XkyZAQAaGO9i3on-b3fswg', 0.6326159834861755), (u'qdTtkZVgcdu3SEA6tzBPdw', 0.6320397853851318), (u'L7eGNKkuy_XdQ_35Y1Kacg', 0.629540741443634), (u'cd46siFt_-08j9-kSbVEgA', 0.6246991157531738), (u'3mp5jXdxC2yqSK6sgRQfEg', 0.6225901246070862)] 

[('draft', 0.7733069062232971), (u'wine', 0.7592223882675171), ('tap', 0.7554447054862976), ('micro-brews', 0.7395221590995789), ('brew', 0.7344582080841064), (u'cocktail', 0.7317838072776794), ('draught', 0.7313833236694336), ('selection', 0.723392128944397), ('bottled', 0.71184903383255), ('whisky', 0.6925665140151978)] 

[(u'W3SROyBvrFKT5C2ySdx1qw', 0.44849711656570435), (u'6w6gMZ3iBLGcUM4RBIuifQ', 0.43213915824890137), (u'KCP4tSmVRD6Gk3xbPEAf3w', 0.3971535861492157), (u'3WsATGkAIXV-56eWjdzecw', 0.38